# 2020 Data
### LMI Capstone Team
### Summer Chambers | Steve Morris | Kaleb Shikur

## Setup

In [1]:
import pandas as pd
import numpy as np
import json
import math
from statistics import mode
from sklearn.metrics import ndcg_score, f1_score, average_precision_score
from matplotlib import pyplot as plt
import seaborn as sns
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
import regex as re
from bs4 import BeautifulSoup
import requests #gets urls
import time
import csv
import os
import sys
import string
from random import sample
from sklearn.metrics import f1_score as f1, confusion_matrix as confusion, plot_roc_curve as roc
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import plot_importance
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer

In [2]:
# 173.79.72.92, port 9200
# host = '173.79.72.92'
endpoint = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com'

In [3]:
#es = Elasticsearch(endpoint, timeout = 45)
es = Elasticsearch(endpoint, timeout=600, max_retries=2, retry_on_timeout=True)



print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'eb7b3348d3c34ce7dc22f263930c51ad', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


## True Sections

In [305]:
metadata_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/CMS-2020-0088-1604%20Comment%20MetaData.xlsx"
data = pd.read_excel(metadata_url)
data = data.replace(["EUC Policy for 2021", "Comment Solicitation: EUC Policy for 2022 and beyond"], \
             "Comment Solicitation: EUC Policy for 2021 and beyond") #combining two sections that can't be split in rule
data = data.loc[data.Topic.notnull()]
data["comment_id"] = [i for i, comment in enumerate(data.Primary)]

In [306]:
data

,Primary,Tracking Number,Organization,Topic,Supportive?,Comment text,Unique Flag,HeavyHitter,Modified,Form Letter?,FormLetter Y/N,Sheet Name,comment_id
0,CMS-2020-0088-9495,1k4-9ivz-162j,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,NaN,Liz Tracker,0
1,CMS-2020-0088-9494,1k4-9ivz-50bx,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,NaN,Liz Tracker,1
2,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Applying the APM APP to SSP ACOs,Opposed,Thank you for the opportunity to provide publi...,1,NaN,2020-10-09 11:23:30,NaN,NaN,Dylan Tracker,2
3,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Revising the SSP Quality Performance Standard,Opposed,"...In addition, the move to a 4th decile ACO q...",0,NaN,2020-10-09 11:23:30,NaN,NaN,Dylan Tracker,3
4,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Applying the APM APP to SSP ACOs,Opposed,We respectfully decline from supporting CMS pr...,1,NaN,2020-10-08 18:18:49,NaN,NaN,Eric Tracker,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,CMS-2020-0088-10352,kf2-z7zt-wvsk,American College of Cardiology (ACC),Other: Unsolicited,Other - Unsolicited,"In addition, the ACC requests more clarity on ...",0,NaN,2020-09-23 17:46:39,NaN,NaN,Eric Tracker,528
531,CMS-2020-0088-10352,kf2-z7zt-wvsk,American College of Cardiology (ACC),Applying the APM APP to SSP ACOs,Mixed,Phase-In of APP\n\nCMS proposes to require the...,0,NaN,2020-10-09 11:25:29,NaN,NaN,Eric Tracker,529
532,CMS-2020-0088-0075,kdt-1riv-ywkl,Mary Washington Health Alliance,Updating Primary Care Services Definition for ...,Supportive,I am the Medical Director of the Mary Washingt...,1,NaN,2020-08-14 15:29:04,NaN,NaN,Dylan Tracker,530
533,CMS-2020-0088-0029,kdj-2v1r-x4wk,Cynthia Munshell,Other: Unsolicited,Other - Unsolicited,(UNCLEAR IF SSP) I am writing to recommend tha...,1,NaN,2020-08-14 15:20:52,NaN,NaN,Dylan Tracker,531


### Topic Count

In [210]:
data.Topic.value_counts()

Applying the APM APP to SSP ACOs                                                             245
Revising the SSP Quality Performance Standard                                                 71
Updating Primary Care Services Definition for Assignment                                      48
Comment Solicitation: EUC Policy for 2020                                                     41
Proposed Changes to CAHPS for 2020                                                            33
Comment Solicitation: EUC Policy for 2021 and beyond                                          29
Other: Unsolicited                                                                            25
Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses     16
Revising Policy for Determining the Amount of Repayment Mechanism                             15
Applicability of Policies to Track 1+ Model ACOs                                               4
Quality Redesign: Compliance w

## Rule-Splitting

In [211]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/Rule/CMS-2020-0088-1604.txt"

In [212]:
startdict = {"Applying the APM APP to SSP ACOs": ["b. Applying the Alternative Payment Model (APM) Performance Pathway (APP) to Shared Savings Program ACOs", \
                                                  "c. Shared Savings Program Quality Performance Standard"], \
           "Revising the SSP Quality Performance Standard": ["c. Shared Savings Program Quality Performance Standard", \
                                                             "d. Use of ACO Quality Performance in Determining Shared Savings and Shared Losses"], \
           "Comment Solicitation: EUC Policy for 2020": ["2. Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020", \
                                                         "J. Proposal To Remove Selected National Coverage Determinations"], \
           "Updating Primary Care Services Definition for Assignment": ["2. Revisions to the Definition of Primary Care Services used in Shared Savings Program Beneficiary Assignment", \
                                                                        "3. Reducing the Amount of Repayment Mechanisms for Eligible ACOs"], \
           "Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses": ["d. Use of ACO Quality Performance in Determining Shared Savings and Shared Losses", \
                                                                                                         "e. Compliance With the Quality Performance Standard"], \
           "Quality Redesign: Compliance with the Quality Performance Standard": ["e. Compliance With the Quality Performance Standard", \
                                                                                  "f. Updating the Process Used To Validate ACO Quality Data Reporting"], \
           "Proposed Changes to CAHPS for 2020": ["1. Proposed Changes to the CAHPS for ACOs Reporting Requirements for Performance Year 2020", \
                                                  "2. Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020"], \
           "Applicability of Policies to Track 1+ Model ACOs": ["4. Applicability of Policies to Track 1+ Model ACOs", \
                                                                "I. Modifications to Quality Reporting Requirements and Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020"], \
           "Revising Policy for Determining the Amount of Repayment Mechanism": ["3. Reducing the Amount of Repayment Mechanisms for Eligible ACOs", \
                                                                                 "4. Applicability of Policies to Track 1+ Model ACOs"], \
           "Quality Redesign: Proposed Technical Changes to Quality Regulation Text":  ["h. Proposed Technical Changes To Incorporate References to Revised Quality Performance Standard", \
                                                                                        "2. Revisions to the Definition of Primary Care Services Used in Shared Savings Program Beneficiary Assignment"], \
           "Comment Solicitation: EUC Policy for 2021 and beyond": ["g. Changes to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2021", \
                                                                    "h. Proposed Technical Changes To Incorporate References to Revised Quality Performance Standard"], \
           "Updating the Process Used to validate ACO Quality Data Reporting": ["f. Updating the Process Used To Validate ACO Quality Data Reporting", \
                                                                                "g. Changes to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2021"] }

## Headers Split

In [213]:
def splitRule_headers(rule_url, startdict):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    for key in startdict.keys():
        startdict[key] = [val.lower() for val in startdict[key]]
    
    rulechunks = []
    
    for key, value in startdict.items():    
       splitlist = alltxt.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks.append({"section": key, "text": (value[0]+split_further[0])}) #choose only first half to upload to dict

    return rulechunks

In [214]:
split1 = splitRule_headers(rule_url, startdict)
headers_id_list = [doc["section"] for doc in split1]
lengths = {chunk["section"]: len(chunk["text"]) for chunk in split1}
print("sections:", len(lengths))
lengths

sections: 12


{'Applying the APM APP to SSP ACOs': 23299,
 'Revising the SSP Quality Performance Standard': 7592,
 'Comment Solicitation: EUC Policy for 2020': 10933,
 'Updating Primary Care Services Definition for Assignment': 45692,
 'Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses': 9614,
 'Quality Redesign: Compliance with the Quality Performance Standard': 18527,
 'Proposed Changes to CAHPS for 2020': 8890,
 'Applicability of Policies to Track 1+ Model ACOs': 9995,
 'Revising Policy for Determining the Amount of Repayment Mechanism': 22281,
 'Quality Redesign: Proposed Technical Changes to Quality Regulation Text': 3041,
 'Comment Solicitation: EUC Policy for 2021 and beyond': 11322,
 'Updating the Process Used to validate ACO Quality Data Reporting': 2638}

## Hybrid Split

In [215]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = []
    chunks = splitRule_headers(rule_url, startdict)
    for doc in chunks:
        paragraphs = doc["text"].split('\r\n')
        #add new lines while under 6000 characters
        for i in range(len(paragraphs) - 1):
            while i < (len(paragraphs)-1) and len(paragraphs[i]) < 6000:
                paragraphs[i] += paragraphs[i+1]
                del(paragraphs[i+1])
        for i in range(len(paragraphs)):
            new_rule_chunks.append({"section": doc["section"]+"_"+str(i), "text": paragraphs[i]})
    return new_rule_chunks

In [216]:
split2 = splitRule_line_hybrid(rule_url)
hybrid_id_list = [doc["section"] for doc in split2]
lengths = {chunk["section"]: len(chunk["text"]) for chunk in split2}
print("sections:", len(lengths))
lengths

sections: 32


{'Applying the APM APP to SSP ACOs_0': 6267,
 'Applying the APM APP to SSP ACOs_1': 8189,
 'Applying the APM APP to SSP ACOs_2': 6735,
 'Applying the APM APP to SSP ACOs_3': 2046,
 'Revising the SSP Quality Performance Standard_0': 6873,
 'Revising the SSP Quality Performance Standard_1': 703,
 'Comment Solicitation: EUC Policy for 2020_0': 6352,
 'Comment Solicitation: EUC Policy for 2020_1': 4557,
 'Updating Primary Care Services Definition for Assignment_0': 6489,
 'Updating Primary Care Services Definition for Assignment_1': 6307,
 'Updating Primary Care Services Definition for Assignment_2': 6107,
 'Updating Primary Care Services Definition for Assignment_3': 8129,
 'Updating Primary Care Services Definition for Assignment_4': 7356,
 'Updating Primary Care Services Definition for Assignment_5': 6711,
 'Updating Primary Care Services Definition for Assignment_6': 4391,
 'Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses_0': 6169,
 'Quality Re

In [ ]:
split2

## Upload Splits to ES

In [13]:
##es.indices.delete(index='2020headers_1shard', ignore=[400, 404])

In [11]:
def rulesplit_toES(rulechunks, es_index):
    for chunk in rulechunks:
            res = es.index(index=es_index, id=chunk["section"], body=chunk, doc_type='_doc')
            es.indices.refresh(index=es_index)
    print("Last id uploaded:", chunk["section"])

In [12]:
rulesplit_1shard = splitRule_headers(rule_url, startdict)
upload_1shard = rulesplit_toES(rulesplit_1shard, "2020headers_1shard")

Last id uploaded: Updating the Process Used to validate ACO Quality Data Reporting


In [346]:
rulesplit_1shard = splitRule_line_hybrid(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "2020hybrid_narrowsyn_custstop")

Last id uploaded: Updating the Process Used to validate ACO Quality Data Reporting_0


## Term Vectors 

#### Get top common words

In [217]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 30,
    "min_term_freq": 1,
    "min_doc_freq": 11 ## 90% of documents
  }
}

In [218]:
termvecs = dict()
for section in headers_id_list:
    result = es.termvectors(index = "2020headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [219]:
common_words = [value for key, value in termvecs.items()]
common_words = list(set([word for list1 in common_words for word in list1]))
common_words

['cms',
 '3',
 'subsequent',
 'eligible',
 'requirements',
 '1',
 'year',
 'performance',
 'proposed',
 'program',
 'data',
 'shared',
 'current',
 'savings',
 'years',
 'aco',
 'acos',
 '2',
 'period',
 '2021',
 'beginning']

#### Examine top words

In [220]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 10,
    "min_term_freq": 1,
    "min_doc_freq": 1,
    "max_doc_freq": 10
  }
}

In [221]:
termvecs = dict()
for section in headers_id_list:
    result = es.termvectors(index = "2020headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [222]:
for key, value in termvecs.items():
    print(key, "\n", value, "\n\n")

Applying the APM APP to SSP ACOs 
 ['app', 'category', 'measure', 'measures', 'mips', 'quality', 'report', 'reporting', 'scoring', 'set'] 


Revising the SSP Quality Performance Standard 
 ['40th', 'category', 'facility', 'mips', 'percentile', 'quality', 'score', 'scores', 'scoring', 'standard'] 


Comment Solicitation: EUC Policy for 2020 
 ['2020', 'circumstances', 'completely', 'extreme', 'measures', 'phe', 'quality', 'report', 'score', 'uncontrollable'] 


Updating Primary Care Services Definition for Assignment 
 ['assignment', 'care', 'code', 'codes', 'cpt', 'management', 'primary', 'service', 'services', 'snf'] 


Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses 
 ['d', 'enhanced', 'level', 'loss', 'maximum', 'percent', 'quality', 'rate', 'sided', 'track'] 


Quality Redesign: Compliance with the Quality Performance Standard 
 ["aco's", 'agreement', 'consecutive', 'entering', 'failed', 'meet', 'participation', 'quality', 'standard', 'term

## EXCEL COMMENTS

In [307]:
excel2020 = {num:text for num, text in zip(data["comment_id"], data["Comment text"])}

### Write to .json

In [310]:
dump = json.dumps(excel2020)
f = open('excel2020.json','w')
f.write(dump)
f.close()

## Remove duplicate excel comments

In [311]:
list_excel2020 = list(excel2020.values())

In [312]:
from DeDuper import *
duplicates = getDupes(list_excel2020)
print(len(duplicates))

96


### Examine Duplicates

In [313]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_excel2020[idx][0:120] for idx in dup]

In [314]:
#dupdict

### Remove Them

In [315]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_excel2020):
        if idx in duplist[:-1]:
            tb_deleted.append(idx)

In [316]:
len(tb_deleted)

239

In [317]:
unique_comments2020 = {key:value for key, value in excel2020.items() if key not in tb_deleted}

In [318]:
len(unique_comments2020)

294

## Create Truth Dictionary

In [322]:
dup_topics = {}
for dup_list in duplicates: #iterate through the duplicate lists
    topic_list = []
    for dup in dup_list: #iterate through the comments
        topic_list.append(data.loc[data["comment_id"]==dup, 'Topic'].iloc[0]) #find topic for that comment
    dup_topics[int(dup)] = list(set(topic_list)) #choose last comment in list, map it to only unique topics in that duplicates list
unique_only = [idx for idx in unique_comments2020.keys() if idx not in dup_topics.keys()]
for unique in unique_only:
    dup_topics[int(unique)] = [data.loc[data['comment_id']==unique, 'Topic'].iloc[0]]

In [323]:
truth_dict = dict(sorted(dup_topics.items()))

In [324]:
len(truth_dict)

294

### Write truth_dict

In [325]:
dump = json.dumps(truth_dict)
f = open('truth_dict2020.json','w')
f.write(dump)
f.close()

### Read truth_dict (avoid running duplicates)

In [326]:
with open('truth_dict2020.json') as f:
    truth_dict = json.load(f)
    truth_dict = {int(key):value for key, value in truth_dict.items()}

### Create Unique DataFrame with topics as columns, ones and zeroes

In [332]:
all_unique_data = data.loc[data['comment_id'].isin(truth_dict.keys())]
topics = list(all_unique_data.Topic.unique())

In [335]:
topic_df = all_unique_data.copy()
for topic in topics:
    topic_df[topic] = np.zeros(len(all_unique_data.Topic))
topic_df

,Primary,Tracking Number,Organization,Topic,Supportive?,Comment text,Unique Flag,HeavyHitter,Modified,Form Letter?,...,Revising the SSP Quality Performance Standard,Comment Solicitation: EUC Policy for 2020,Comment Solicitation: EUC Policy for 2021 and beyond,Updating Primary Care Services Definition for Assignment,Other: Unsolicited,Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses,Quality Redesign: Compliance with the Quality Performance Standard,Proposed Changes to CAHPS for 2020,Revising Policy for Determining the Amount of Repayment Mechanism,Applicability of Policies to Track 1+ Model ACOs
0,CMS-2020-0088-9495,1k4-9ivz-162j,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Applying the APM APP to SSP ACOs,Opposed,Thank you for the opportunity to provide publi...,1,NaN,2020-10-09 11:23:30,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Revising the SSP Quality Performance Standard,Opposed,"...In addition, the move to a 4th decile ACO q...",0,NaN,2020-10-09 11:23:30,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Applying the APM APP to SSP ACOs,Opposed,We respectfully decline from supporting CMS pr...,1,NaN,2020-10-08 18:18:49,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Applying the APM APP to SSP ACOs,Opposed,PBACO is in favor of reducing administrative b...,0,NaN,2020-10-08 18:18:49,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,CMS-2020-0088-11199,1k4-9j18-zaey,Connected Health Initiative,Other: Unsolicited,Other - Unsolicited,"To date, CMS has not discussed digital health ...",1,NaN,2020-09-24 12:33:17,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,CMS-2020-0088-10979,kf5-rva9-6mog,Association for Clinical Oncology,Comment Solicitation: EUC Policy for 2020,Supportive,ASCO supports the agency’s proposals for flexi...,0,NaN,2020-09-22 10:23:47,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
530,CMS-2020-0088-10352,kf2-z7zt-wvsk,American College of Cardiology (ACC),Other: Unsolicited,Other - Unsolicited,"In addition, the ACC requests more clarity on ...",0,NaN,2020-09-23 17:46:39,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
533,CMS-2020-0088-0029,kdj-2v1r-x4wk,Cynthia Munshell,Other: Unsolicited,Other - Unsolicited,(UNCLEAR IF SSP) I am writing to recommend tha...,1,NaN,2020-08-14 15:20:52,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [337]:
for key, value in truth_dict.items():
    for topic in topics:
        if topic in value:
            topic_df[topic][key] = 1
topic_df[topics] = topic_df[topics].astype(int)

<ipython-input-337-aaa36271f3c5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_df[topic][key] = 1


In [339]:
topic_df[topics].sum()

Applying the APM APP to SSP ACOs                                                             75
Revising the SSP Quality Performance Standard                                                25
Comment Solicitation: EUC Policy for 2020                                                    17
Comment Solicitation: EUC Policy for 2021 and beyond                                          7
Updating Primary Care Services Definition for Assignment                                     14
Other: Unsolicited                                                                            8
Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses     6
Quality Redesign: Compliance with the Quality Performance Standard                            3
Proposed Changes to CAHPS for 2020                                                           10
Revising Policy for Determining the Amount of Repayment Mechanism                             6
Applicability of Policies to Track 1+ Mo

## Get only unique comments text, make lowercase, replace \t etc.

In [340]:
unique_excel2020 = {key:value.lower().replace('\t', " ").replace('\n', " ").replace('\r', " ").replace('\s', " ")\
                    for key, value in excel2020.items() if key in list(truth_dict.keys())}

### Write unique comments

In [341]:
dump = json.dumps(unique_excel2020)
f = open('unique_excel2020.json','w')
f.write(dump)
f.close()

### Read unique comments (avoid running duplicates)

In [342]:
with open('unique_excel2020.json') as f:
    unique_excel2020 = json.load(f)
    unique_excel2020 = {int(key):value for key, value in unique_excel2020.items()}

## Querying ES

In [343]:
#Single query
def ES_search(es_index, querydict, search_type=None, size=10):
    if search_type:
        search = es.search(index=es_index, doc_type="_doc", body={"query": querydict}, search_type=search_type, size=size)
    else:
        search = es.search(index=es_index, doc_type="_doc", body={"query": querydict}, size=size)
    test_dict = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            test_dict[h['_id']]=h['_score']
    return test_dict

### Querying Many Comments

In [344]:
def run_large_query(comment_dict, es_index, size=10):
    results = {}
    for key, value in comment_dict.items():
        query = {"match" : {"text" : value}}
        try:
            search1 = ES_search(es_index, query, 'dfs_query_then_fetch', size=size)
        except ElasticsearchException as es1:
            print(es1)
            es.indices.refresh(index=es_index)
            continue
        results[key] = {section:score for section, score in search1.items()}
        time.sleep(3)
        es.indices.refresh(index=es_index)
        print(key, end=' ')
    return results

In [348]:
hybrid_custom_results = run_large_query(unique_excel2020, "2020hybrid_narrowsyn_custstop", size=20)

0 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 28 30 32 33 34 35 40 41 43 44 45 46 47 48 49 50 52 53 54 55 56 57 58 59 60 61 62 63 64 66 67 69 72 73 74 75 79 80 82 83 85 86 88 91 92 93 96 97 98 104 105 106 107 108 109 110 111 112 113 114 117 118 119 120 121 122 123 126 128 130 133 134 136 137 138 139 141 142 143 144 145 146 147 148 151 152 153 154 155 158 159 160 162 163 164 165 166 171 172 174 175 176 178 179 180 181 183 184 185 186 187 188 191 195 196 199 200 202 203 206 207 208 209 210 211 212 213 214 215 216 217 218 219 222 228 229 230 231 236 237 242 243 244 245 247 250 251 252 253 254 255 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 278 280 282 284 286 290 291 292 293 294 295 297 298 299 300 301 302 303 304 305 306 307 308 312 313 314 315 317 320 324 325 327 330 333 334 335 337 339 340 342 343 344 346 347 348 351 352 354 357 361 364 369 371 372 373 376 377 378 379 380 381 382 384 391 393 394 396 400 409 412 415 420 421 422 426 

### Write to .json

In [349]:
dump = json.dumps(hybrid_custom_results)
f = open('2020hybrid_narrowsyn_custstop.json','w')
f.write(dump)
f.close()

# Defining a Score_Results Function

In [350]:
#Kaleb's MAP Function
def Map_estimator(tru, pred): 
    precision = []
    for x,y in zip(tru,pred):
        wow = 0
        counter = 0
        for i, z in enumerate(y):
            if z in x:
                counter += 1
                wow += 1/(i+1)
            if i >= 1 + len(x):
                break;
        precision.append(wow/(counter+0.0001))
    return np.mean(precision)

In [382]:
def score_results(json_file, index_type): #takes .json result and "headers"/"hybrid" as input, gives pandas eval df and map as output
    #open file
    with open(json_file) as f:
        custom_results = json.load(f)
    
    if index_type == "hybrid":
        #tranlate ES keys to true rule section names, MAINTAINING ORDER
        for key, value in custom_results.items():
            value_list = []
            for val_idx in value:
                for true_key in truth_keys:
                    if true_key in val_idx[0:-2]:
                        value_list.append(true_key)
                custom_results[key]=[(true_key, score) for (val_idx, score), true_key in zip(value.items(),value_list)]
        for comment_idx, tup_list in custom_results.items():
            score_dict = {}
            for tup in tup_list:
                if tup[0] in score_dict.keys():
                    score_dict[tup[0]] += tup[1]
                else:
                    score_dict[tup[0]] = tup[1]
            custom_results[comment_idx] = score_dict
    

    elif index_type == "headers": 
        #tranlate ES keys to true rule section names, MAINTAINING ORDER
        for key, value in custom_results.items():
            value_list = []
            for val_idx, score in value.items():
                for true_key in truth_keys:
                    if true_key in val_idx and true_key not in value_list: 
                        value_list.append(true_key)
                custom_results[key]={true_key: score for (val_idx, score), true_key in zip(value.items(),value_list)}     
    
    #Normalize results by dividing by mean
    normalized_results = {}
    for comment_idx, result in custom_results.items():
        score_mean = np.mean(list(result.values()))
        score_dict = {}
        for rule_idx, score in result.items():
            score_dict[rule_idx] = score/score_mean
        normalized_results[comment_idx] = score_dict
    normalized_lists = {comment_idx:list(score_dict.items()) for comment_idx, score_dict in normalized_results.items()}

    #Merge results back together
#     merged_results = {}
#     for key in truth_dict.keys():
#         merged_list = []
#         for comment_idx, result_list in normalized_lists.items():
#             print(comment_idx)
#             if key in comment_idx:
#                 merged_list.extend(result_list)
#         merged_results[key] = merged_list
#     merged_results = {comment_idx:sorted(result, key=lambda x:x[1], reverse=True) for comment_idx, result in merged_results.items()}
#     for comment_idx, tup_list in merged_results.items():
#         score_dict = {}
#         for tup in tup_list:
#             if tup[0] in score_dict.keys():
#                 score_dict[tup[0]] += tup[1]
#             else:
#                 score_dict[tup[0]] = tup[1]
#         merged_results[comment_idx] = score_dict
#     merged_results = {comment_idx:dict(sorted(result.items(), key = lambda x:x[1], reverse=True)) for comment_idx, result in merged_results.items()}
    
    merged_results = {comment_idx:dict(sorted(result.items(), key = lambda x:x[1], reverse=True)) for comment_idx, result in normalized_results.items()}
    
    #Remove frequently returned rule sections (in >80% of results)
    all_sections = []
    for key, result in merged_results.items():
        all_sections.extend(section for section in result)
    counts = {}
    for key in truth_keys:
        key_count = all_sections.count(key)
        counts[key] = key_count
    tb_removed = [section for section, count in counts.items() if count > (.8*len([len(value) for value in truth_dict.values()]))]
    
    def comm_removed(merged_dict, tb_removed):
        new_merged = {}
        for com_id, x in merged_dict.items():
            results = {}
            for key, value in x.items():
                if key not in tb_removed:
                    results[key] = value 
                elif key in tb_removed:
                    results[key] = value*.25
                new_merged[com_id] = results
        return new_merged
    
    clean_merged = comm_removed(merged_results, tb_removed)
    clean_merged = {comment_idx:dict(sorted(result.items(), key = lambda x:x[1], reverse=True)) for comment_idx, result in clean_merged.items()}
    
    #create truth dict with values equal to sparse array length=29, binary
    truth_dict_all = truth_dict.copy()   #make a copy of the truth dictionary
    for key, value in truth_dict.items(): #for true key/val pair in dictionary
        new_value_list = np.zeros(len(truth_keys)) #make array of zeros, of length 29 (29 rule sections)
        for val in value: #for each rule section
            for i in range(len(truth_keys)): #for each of 29 spots in array
                if val == truth_keys[i]: #if rule section in current array index,
                    new_value_list[i] = 1 #set to 1, else 0
        truth_dict_all[key] = new_value_list
    
    #create header results dict with values equal to sparse array length=29, (discounted and binary versions)
    custom_results_all = clean_merged.copy() #make copy of results
    custom_binary_results_all = clean_merged.copy() #make copy of results
    for key, value in clean_merged.items(): #for each comment
        new_value_list_discounted = np.zeros(len(truth_keys)) #make array of zeros, of length 29 (29 rule sections)
        new_value_list_binary = np.zeros(len(truth_keys)) #make array of zeros, of length 29 (29 rule sections)
        counter = 1 #initiate counter for a query result at 1
        for rule_section in value.keys(): #go through all rule sections in query
            for i in range(len(truth_keys)): #for each of 29 spots in array
                if rule_section == truth_keys[i]: #if rule section in current array index
                    counter += 1 #add to counter
                    #assign value to 1 for first rule section, but penalize lower scored rule sections with scores progressively lower than one
                    new_value_list_discounted[i] = 1/math.log(counter, 2) 
                    new_value_list_binary[i] = 1 #binary means we lose the order ES returned rules in, just weights all sections equally
        custom_results_all[key] = new_value_list_discounted
        custom_binary_results_all[key] = new_value_list_binary
    
    #Get NDCG, F1 with SKlearn
    custom_ndcg = {}
    custom_f1 = {}
    for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), custom_results_all.items()):
        custom_ndcg[true_key] = ndcg_score([true_value], [es_value])
    for (true_key, true_value), (es_key, es_value) in zip(truth_dict_all.items(), custom_binary_results_all.items()):
        custom_f1[true_key] = f1_score(true_value, es_value, zero_division=0)
    df_eval = pd.DataFrame(data=[custom_ndcg, custom_f1]).T
    df_eval = df_eval.rename({0:"NDCG", 1:"F1"}, axis=1)
    
    ap_results = Map_estimator(list(truth_dict.values()), list(clean_merged.values()))
    
    return df_eval, ap_results

In [386]:
truth_keys = topics
print("MAP:", score_results('2020hybrid_narrowsyn_custstop.json', "hybrid")[1])
np.mean(score_results('2020hybrid_narrowsyn_custstop.json', "hybrid")[0])

MAP: 0.5897763479379716


NDCG    0.719112
F1      0.219450
dtype: float64

# Classification

### Train/Test Split

In [387]:
train, test = train_test_split(topic_df, test_size=0.2, random_state=44)

In [388]:
train_texts = list(train["Comment text"])
test_texts = list(test["Comment text"])

In [389]:
train.sum(axis=0)[topics]

Applying the APM APP to SSP ACOs                                                             55
Revising the SSP Quality Performance Standard                                                21
Comment Solicitation: EUC Policy for 2020                                                    17
Comment Solicitation: EUC Policy for 2021 and beyond                                          6
Updating Primary Care Services Definition for Assignment                                      9
Other: Unsolicited                                                                            6
Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses     6
Quality Redesign: Compliance with the Quality Performance Standard                            2
Proposed Changes to CAHPS for 2020                                                           10
Revising Policy for Determining the Amount of Repayment Mechanism                             6
Applicability of Policies to Track 1+ Mo

## Classify All Rule Sections - TFIDF

In [407]:
def stem_tokenize(text):
    tokens = word_tokenize(text)
    stems = [PorterStemmer().stem(item) for item in tokens]
    return stems

tfidf_vector = TfidfVectorizer(tokenizer=stem_tokenize, ngram_range=(1,1), max_df=0.8)
x_train = tfidf_vector.fit_transform(train_texts)
y_train = np.array(train[topics])

x_test = tfidf_vector.transform(test_texts)
y_test = np.array(test[topics])

In [408]:
boost = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

multi_boost = MultiOutputClassifier(boost)

multi_boost.fit(x_train, y_train)

multi_boost_preds = multi_boost.predict(x_test)

In [409]:
multi_boost_f1 = f1(y_test, multi_boost_preds, zero_division=0, average=None)

In [410]:
scores = {sec:score for (sec, score) in zip(topics, list(multi_boost_f1))}
scores

{'Applying the APM APP to SSP ACOs': 0.5,
 'Revising the SSP Quality Performance Standard': 0.0,
 'Comment Solicitation: EUC Policy for 2020': 0.0,
 'Comment Solicitation: EUC Policy for 2021 and beyond': 0.0,
 'Updating Primary Care Services Definition for Assignment': 0.28571428571428575,
 'Other: Unsolicited': 0.0,
 'Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses': 0.0,
 'Quality Redesign: Compliance with the Quality Performance Standard': 0.0,
 'Proposed Changes to CAHPS for 2020': 0.0,
 'Revising Policy for Determining the Amount of Repayment Mechanism': 0.0,
 'Applicability of Policies to Track 1+ Model ACOs': 0.0}

In [411]:
np.mean(multi_boost_f1)

0.07142857142857144

#### Identifying Most Significant Words for Classification

In [405]:
features = []
for boost in multi_boost.estimators_:
    boost_top10 = np.argsort(-boost.feature_importances_)[0:10]
    features.append([(tfidf_vector.get_feature_names()[feature]) for feature in boost_top10])

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:749: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()


In [406]:
important_features = {sec:feature_list for (sec, feature_list) in zip(topics, features)}
boost_features = {key:(value,features) for (key, value), (key1, features) in zip(scores.items(), important_features.items())}

boost_features

{'Applying the APM APP to SSP ACOs': (0.5,
  ['howev',
   'how',
   'over',
   '23',
   'data',
   'calcul',
   'and',
   'least',
   'pathway',
   'sunset']),
 'Revising the SSP Quality Performance Standard': (0.0,
  ['30th',
   'against',
   'veri',
   'complianc',
   'success',
   'at',
   'implement',
   'address',
   'should',
   '40th']),
 'Comment Solicitation: EUC Policy for 2020': (0.0,
  ['have',
   'uncontrol',
   'own',
   'measur',
   '2020',
   'are',
   'extrem',
   '-',
   'emerg',
   'py']),
 'Comment Solicitation: EUC Policy for 2021 and beyond': (0.0,
  ['affect',
   'circumst',
   'given',
   'survey',
   'further',
   'uncontrol',
   'mssp',
   '2021',
   'may',
   'chang']),
 'Updating Primary Care Services Definition for Assignment': (0.28571428571428575,
  ['expand',
   'definit',
   'assign',
   ',',
   'code',
   'therefor',
   'support',
   'is',
   'and',
   'are']),
 'Other: Unsolicited': (0.0,
  ['retain',
   'consider',
   'activ',
   'high',
   'made',
 

## SCRAPE COMMENTS

In [100]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/Comments/"

In [107]:
response = requests.get(comment_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tags = soup.findAll("a")
links = [tag["href"] for tag in a_tags]
txt_links = [link for link in links if '.txt' in link]
comments = {}
txt_sampled = sample(txt_links, 400)
for suffix in txt_sampled:
    comments[suffix[14:19]] = requests.get(comment_url+suffix).text.lower()
    print(suffix[14:19], end=" ")

27263 14757 4081. 12407 9318. 7951_ 6346. 22950 25087 10310 19016 11720 5169. 11847 26170 16844 19308 5615_ 16732 29378 10132 21984 9460. 31050 27775 24212 5148. 23098 5446. 30719 20607 18248 3580. 6017. 17690 3873. 16401 11983 7781. 29013 30443 25489 13666 7303. 28679 26773 14237 26913 27652 2383. 29581 4959. 7359. 6914. 14712 7314. 6766. 17430 2725. 30465 22797 22507 2862. 11882 19051 1740. 26872 21773 13936 7426_ 12483 17048 8911. 24176 3100. 25565 21589 4851. 28195 4280. 2722. 25510 20831 22954 3805. 22975 27428 7039. 21402 28609 12890 26777 4139. 25190 23669 16279 20372 23417 18884 15849 26058 7196. 31315 15342 17861 20653 12522 23079 6932. 20846 24710 31121 16031 18438 2271. 29538 28145 2571. 12597 8130. 5170. 21016 21611 19436 10114 23130 28470 28359 7163. 24545 8714. 24722 12659 6621. 25738 29803 25459 23234 12218 21052 23291 7433. 25182 25817 12374 15718 23309 16215 8790. 5153. 9578_ 22221 8782. 22024 26763 11121 29343 14207 15870 3601. 4388. 13142 10516 13236 29831 16123 4046

### Write to .json

In [120]:
comment_json = json.dumps(comments)
f = open('comments2020.json','w')
f.write(comment_json)
f.close()

### Read .json (avoid scraping comments)

In [121]:
with open('comments2020.json') as f:
    comments2020 = json.load(f)

## Remove duplicate comments, comments less than 30 characters long

In [124]:
reduced_comments2020 = {key: val for key, val in comments2020.items() if len(val) >= 30}
short_ones = {key: val for key, val in comments2020.items() if len(val) < 30}
list_reduced_comments2020 = list(reduced_comments2020.values())
len(reduced_comments2020)
short_ones #these seem okay to remove

{'26913': '\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n',
 '22797': 'please see attached letter',
 '8911.': 'see attached file(s)',
 '24176': 'see attached file(s)',
 '28195': 'see attached file(s)',
 '22954': 'see attached file(s)',
 '22975': 'please see attached file.',
 '31315': 'see attached',
 '2571.': 'see attached file(s)',
 '13590': 'see attached file(s)',
 '24553': 'see attached file(s)',
 '19450': 'see attached file(s)',
 '20209': 'see attached file(s)',
 '13417': 'please see attached file.',
 '14666': 'see attached file(s)',
 '30264': 'my letter is attached.',
 '10753': 'see attached file(s)',
 '14093': 'see attached',
 '14129': 'see attached file(s)'}

In [126]:
from DeDuper import *
duplicates = getDupes(list_reduced_comments2020)
print(len(duplicates))
#duplicates

23


### Examine Duplicates

In [128]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_reduced_comments2020[idx][0:120] for idx in dup]

In [130]:
#dupdict

### Remove Them

In [131]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_reduced_comments2020):
        if idx in duplist[:-1]:
            tb_deleted.append(comment)

In [132]:
len(tb_deleted)

306

In [133]:
unique_comments2020 = {key:value for key, value in reduced_comments2020.items() if value not in tb_deleted}

In [134]:
len(unique_comments2020)

68

### Write to .json

In [135]:
dump = json.dumps(unique_comments2020)
f = open('unique_comments2020.json','w')
f.write(dump)
f.close()

### Read .json (avoid running duplicates)

In [136]:
with open('unique_comments2020.json') as f:
    unique_comments2020 = json.load(f)

## Split up longer comments

In [137]:
long_comments = {key:value for key, value in unique_comments2020.items() if len(value) > 5000}

In [139]:
expanded2020 = unique_comments2020.copy()
for key, value in long_comments.items():
    paragraphs = value.split('\n')
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs) - 1) and len(paragraphs[i]) < 4800:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])
    for i in range(len(paragraphs)):
        expanded2020[key+'_'+str(i)] = paragraphs[i]
expanded2020 = {key:value for key, value in expanded2020.items() if key not in list(long_comments.keys())}

In [141]:
#{key:len(value) for key, value in expanded2020.items()}

## Remove \r, \n, \s, and any weird or non-ascii characters

In [142]:
expanded2020 = {key:value.replace("\r", " ").replace("\n", " ").replace("\s", " ") for key, value in expanded2020.items()}

In [143]:
for key, value in expanded2020.items():
    expanded2020[key] = ''.join(c for c in value if c in string.printable)

### Write to .json

In [144]:
dump = json.dumps(expanded2020)
f = open('expanded2020.json','w')
f.write(dump)
f.close()

### Read .json (avoid expanding/cleaning comments)

In [145]:
with open('expanded2020.json') as f:
    expanded2020 = json.load(f)